In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from transformers import pipeline
from src.model_hartmann import load_model as load_hartmann_model, load_tokenizer as load_hartmann_tokenizer
from src.model_custom import load_model as load_custom_model, load_tokenizer as load_custom_tokenizer

In [2]:
hartmann_model = load_hartmann_model()
hartmann_tokenizer = load_hartmann_tokenizer()

custom_model = load_custom_model()
custom_tokenizer = load_custom_tokenizer()

In [3]:
# Create pipelines for easy predictions
hartmann_pipeline = pipeline("text-classification", model=hartmann_model, tokenizer=hartmann_tokenizer, return_all_scores=True)
custom_pipeline = pipeline("text-classification", model=custom_model, tokenizer=custom_tokenizer, return_all_scores=True)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/opt/anaconda3/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
from tabulate import tabulate

goemotions_labels = [
    "admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion", "curiosity",
    "desire", "disappointment", "disapproval", "disgust", "embarrassment", "excitement", "fear",
    "gratitude", "grief", "joy", "love", "nervousness", "optimism", "pride", "realization", "relief",
    "remorse", "sadness", "surprise", "neutral"
]


# Your 10 test sentences
sentences = [
    "I love spending time with my family.",
    "This is the worst day of my life.",
    "I'm feeling very nervous about the exam.",
    "What a beautiful sunset!",
    "I feel so disappointed and frustrated with the situation.",
    "I'm not sure how to feel about this.",
    "That was hilarious, I can't stop laughing!",
    "I feel completely empty and lost.",
    "Your help means a lot to me, thank you!",
    "I'm so angry I could scream."
]

# Loop over sentences and collect results
for i, sentence in enumerate(sentences):
    print(f"========= Sentence {i+1} ==========")
    print(f"Text: {sentence}\n")

    # Get predictions
    hartmann_results = hartmann_pipeline(sentence, return_all_scores=True)
    custom_results = custom_pipeline(sentence, return_all_scores=True)

    # Unwrap the list to get the actual results
    hartmann_results = hartmann_results[0]
    custom_results = custom_results[0]

    # Sort and get top 3 predictions for each
    hartmann_top3 = sorted(hartmann_results, key=lambda x: x['score'], reverse=True)[:3]
    custom_top3 = sorted(custom_results, key=lambda x: x['score'], reverse=True)[:3]

    # Display Hartmann predictions
    print("--- Hartmann Model Top 3 Predictions ---")
    for res in hartmann_top3:
        print(f"{res['label']}: {res['score']:.4f}")

    # Display Custom Model predictions
    print("\n--- Pretrained Model Top 3 Predictions ---")
    for res in custom_top3:
        label_idx = int(res['label'].split("_")[-1])
        emotion = goemotions_labels[label_idx]
        print(f"{emotion}: {res['score']:.4f}")

    print("\n")


========= Sentence 1 ==========
Text: I love spending time with my family.

--- Hartmann Model Top 3 Predictions ---
joy: 0.9883
sadness: 0.0067
disgust: 0.0013

--- Pretrained Model Top 3 Predictions ---
love: 0.9536
joy: 0.0134
admiration: 0.0108


========= Sentence 2 ==========
Text: This is the worst day of my life.

--- Hartmann Model Top 3 Predictions ---
disgust: 0.9805
anger: 0.0086
sadness: 0.0055

--- Pretrained Model Top 3 Predictions ---
anger: 0.3353
surprise: 0.2010
disgust: 0.1235


========= Sentence 3 ==========
Text: I'm feeling very nervous about the exam.

--- Hartmann Model Top 3 Predictions ---
fear: 0.9947
sadness: 0.0013
joy: 0.0011

--- Pretrained Model Top 3 Predictions ---
nervousness: 0.6201
fear: 0.0828
embarrassment: 0.0393


========= Sentence 4 ==========
Text: What a beautiful sunset!

--- Hartmann Model Top 3 Predictions ---
joy: 0.8377
surprise: 0.1189
neutral: 0.0221

--- Pretrained Model Top 3 Predictions ---
admiration: 0.8548
excitement: 0.0729
j